In [ ]:
from sklearn.datasets import load_digits
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Dataset
##### - ข้อมูลที่ใช้
##### - การตรวจสอบข้อมูล
##### - Visualization
    

In [ ]:
# คำอธิบาย Dataset

dataset = load_digits()
print(dataset.DESCR)

In [ ]:
dataset.keys() # จริงๆ แล้วเรามี dataframe ที่ scikit-learn ใช้ได้แล้วตรง dataset.data แล้ว

In [ ]:
dataset.images[0 : 2]

In [ ]:
dataset.images.shape # (1797, 8, 8) เรามีรููปตัวเลขทั้งหมด 1797 แต่ละรูปเป็น 8x8 pixel

In [ ]:
dataset.images[0] # image ของตัวเลขในรูปแบบ Array 2 มิติ

In [ ]:
dataset.target[0] # คำตอบของ image

In [ ]:
plt.imshow(dataset.images[0], cmap='gray_r')
plt.axis('off')
plt.title(f"{dataset.target[0]}")
plt.show()

In [ ]:
i = int(input(f"Enter index 0-{len(dataset.images) -1 }"))
plt.imshow(dataset.images[i], cmap='gray_r')
plt.axis('off')
plt.title(f"{dataset.target[i]}")
plt.show()

In [ ]:
# การเตรียมข้อมูล
df = pd.DataFrame(dataset.images.reshape((len(dataset.images), -1))) # แปลงเป็น 1797 row และมี
df["target"] = dataset.target

In [ ]:
df

In [ ]:
df.isnull().sum() # ไม่มี null

In [ ]:
from sklearn.decomposition import PCA # ใช้ PCA เพื่อดูการจัดกลุ่มข้อมูลเพราะเรามีแกน X 8x8 มิติ และแกน Y 1 มิติ

X = df.drop('target', axis=1)
y = df['target']

pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)

plt.figure(figsize=(10,8))
scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=y, cmap='tab10', alpha=0.7)
plt.legend(*scatter.legend_elements(), title="Digits")
plt.title("PCA Visualization of Digits Dataset")
plt.xlabel("PCA 1")
plt.ylabel("PCA 2")
plt.show() # จะเห็นว่าการกระจายตัวของข้อมูลจะติดๆกัน


In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(6, 3))

axes[0].imshow(dataset.images[5], cmap='gray_r')
axes[0].axis('off')
axes[0].set_title(f"{dataset.target[5]}")

axes[1].imshow(dataset.images[9], cmap='gray_r')
axes[1].axis('off')
axes[1].set_title(f"{dataset.target[9]}")

plt.show() # จะเห็นว่า 5 และ 9 มีความคล้ายกันอยู่

# Train Model and Evaluation
#### - การเลือกโมเดล
#### - การ Train โมเดล
#### - การประเมินโมเดล

In [ ]:
# การแยก data train และ test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) # smaple 20 % มา test โมเดล
print(f"Number of X_train's rows: {X_train.shape[0]}")
print(f"Number of X_test's rows: {X_test.shape[0]}")

In [ ]:
# การเลือกโมเดล KNeighbors เพราะเหมาะสมมากที่สุด
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors=3)

scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')

print("Cross-validation scores:", [f"{score:.3f}" for score in scores])
print(f"Average accuracy: {scores.mean():.3f}") # จะเห็นได้ว่าค่า k = 3 ได้ Average accuracy: 0.985

In [ ]:
avg_scores = []

for k in range(3, 21):
    model = KNeighborsClassifier(n_neighbors=k)
    scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')
    avg_scores.append(scores.mean())

avg_scores

In [ ]:
sns.set_style("darkgrid")
plt.figure(figsize=(10, 5))
plt.bar(range(3, 21), avg_scores)
plt.xlabel("Number of Neighbors (k)")
plt.ylabel("Average Accuracy (5-Fold CV)")
plt.title("KNN Accuracy with Varying k")
plt.xticks(range(1, 21))
margin = 0.1 * (max(avg_scores) - min(avg_scores))
plt.ylim(min(avg_scores) - margin, max(avg_scores) + margin)
plt.show() # ดังกราฟค่า k ที่ดีที่สุดคือ 3

In [ ]:
# Train Model

model = KNeighborsClassifier(n_neighbors=3)
model.fit(X_train, y_train)

In [ ]:
# การประเมินโมเดล
from sklearn.metrics import confusion_matrix

y_pred = model.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
cm

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

print("Accuracy:", round(accuracy_score(y_test, y_pred), 3))
print("Precision:", round(precision_score(y_test, y_pred, average='macro'), 3))
print("Recall:", round(recall_score(y_test, y_pred, average='macro'), 3)) 
print("F1 Score:", round(f1_score(y_test, y_pred, average='macro'), 3))

In [ ]:
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap='Oranges')
plt.show()


In [ ]:
df_test = pd.DataFrame(X_test)
df_test["target"] = y_test
df_test["pred"] = y_pred
df_test

In [ ]:
df_falsePred = df_test.loc[df_test["target"] != df_test["pred"]]
df_falsePred

In [ ]:
plt.figure(figsize=(15, 4))
for i in range(len(df_falsePred)):
    row = df_falsePred.iloc[i]
    
    image = row.iloc[:-2].to_numpy().reshape(8, 8)
    true_label = int(row["target"])
    pred_label = int(row["pred"])
    
    plt.subplot(1, len(df_falsePred), i + 1)
    plt.imshow(image, cmap="gray")
    plt.title(f"True: {true_label}\nPred: {pred_label}")
    plt.axis("off")

plt.tight_layout()
plt.show()

In [ ]:
# เซฟโมเดล
import os
import joblib

os.makedirs('model', exist_ok=True)
joblib.dump(model, './model/digit_recognizer_model.pkl')
